In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2

In [2]:
from tensorflow.keras.datasets import mnist

def load_mnist_dataset():

  # load data from tensorflow framework
  ((trainData, trainLabels), (testData, testLabels)) = mnist.load_data() 

  # Stacking train data and test data to form single array named data
  data = np.vstack([trainData, testData]) 

  # Vertical stacking labels of train and test set
  labels = np.hstack([trainLabels, testLabels]) 

  # return a 2-tuple of the MNIST data and labels
  return (data, labels)

In [3]:
import numpy as np

def load_az_dataset(datasetPath):

  # List for storing data
  data = []
  
  # List for storing labels
  labels = []
  
  for row in open(datasetPath): #Openfile and start reading each row
    #Split the row at every comma
    row = row.split(",")
    
    #row[0] contains label
    label = int(row[0])
    
    image = np.array([int(x) for x in row[1:]], dtype="uint8")
    if image.size < 784:
        image = np.pad(image, (0, 784 - image.size), 'constant')
    image = image.reshape((28, 28))

    
    #append image to data
    data.append(image)
    
    #append label to labels
    labels.append(label)
    
  #Converting data to numpy array of type float32
  data = np.array(data, dtype='float32')
  
  #Converting labels to type int
  labels = np.array(labels, dtype="int")
  
  return (data, labels)

In [4]:

(digitsData, digitsLabels) = load_mnist_dataset()

(azData, azLabels) = load_az_dataset('Data\\A_Z Handwritten Data\\A_Z Handwritten Data.csv')

print(azData.shape)
print(digitsData.shape)


(372451, 28, 28)
(70000, 28, 28)


In [5]:
# the MNIST dataset occupies the labels 0-9, so let's add 10 to every A-Z label to ensure the A-Z characters are not incorrectly labeled 

azLabels += 10

# stack the A-Z data and labels with the MNIST digits data and labels

data = np.vstack([azData, digitsData])
labels = np.hstack([azLabels, digitsLabels])

# Each image in the A-Z and MNIST digts datasets are 28x28 pixels;
# However, the architecture we're using is designed for 32x32 images,
# So we need to resize them to 32x32

data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")

# add a channel dimension to every image in the dataset and scale the
# pixel intensities of the images from [0, 255] down to [0, 1]

data = np.expand_dims(data, axis=-1)
data /= 255.0

In [6]:
le = LabelBinarizer()
labels = le.fit_transform(labels)

counts = labels.sum(axis=0)

# account for skew in the labeled data
classTotals = labels.sum(axis=0)
classWeight = {}

# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
  classWeight[i] = classTotals.max() / classTotals[i]

In [7]:
# construct the image generator for data augmentation

aug = ImageDataGenerator(
rotation_range=10,
zoom_range=0.05,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.15,
horizontal_flip=False,
fill_mode="nearest")

In [8]:
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

class ResNet:
	@staticmethod
	def residual_module(data, K, stride, chanDim, red=False,
		reg=0.0001, bnEps=2e-5, bnMom=0.9):
		# the shortcut branch of the ResNet module should be
		# initialize as the input (identity) data
		shortcut = data

		# the first block of the ResNet module are the 1x1 CONVs
		bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(data)
		act1 = Activation("relu")(bn1)
		conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
			kernel_regularizer=l2(reg))(act1)

		# the second block of the ResNet module are the 3x3 CONVs
		bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(conv1)
		act2 = Activation("relu")(bn2)
		conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
			padding="same", use_bias=False,
			kernel_regularizer=l2(reg))(act2)

		# the third block of the ResNet module is another set of 1x1
		# CONVs
		bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(conv2)
		act3 = Activation("relu")(bn3)
		conv3 = Conv2D(K, (1, 1), use_bias=False,
			kernel_regularizer=l2(reg))(act3)

		# if we are to reduce the spatial size, apply a CONV layer to
		# the shortcut
		if red:
			shortcut = Conv2D(K, (1, 1), strides=stride,
				use_bias=False, kernel_regularizer=l2(reg))(act1)

		# add together the shortcut and the final CONV
		x = add([conv3, shortcut])

		# return the addition as the output of the ResNet module
		return x

	@staticmethod
	def build(width, height, depth, classes, stages, filters,
		reg=0.0001, bnEps=2e-5, bnMom=0.9, dataset="cifar"):
		# initialize the input shape to be "channels last" and the
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# set the input and apply BN
		inputs = Input(shape=inputShape)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(inputs)

		# check if we are utilizing the CIFAR dataset
		if dataset == "cifar":
			# apply a single CONV layer
			x = Conv2D(filters[0], (3, 3), use_bias=False,
				padding="same", kernel_regularizer=l2(reg))(x)

		# check to see if we are using the Tiny ImageNet dataset
		elif dataset == "tiny_imagenet":
			# apply CONV => BN => ACT => POOL to reduce spatial size
			x = Conv2D(filters[0], (5, 5), use_bias=False,
				padding="same", kernel_regularizer=l2(reg))(x)
			x = BatchNormalization(axis=chanDim, epsilon=bnEps,
				momentum=bnMom)(x)
			x = Activation("relu")(x)
			x = ZeroPadding2D((1, 1))(x)
			x = MaxPooling2D((3, 3), strides=(2, 2))(x)

		# loop over the number of stages
		for i in range(0, len(stages)):
			# initialize the stride, then apply a residual module
			# used to reduce the spatial size of the input volume
			stride = (1, 1) if i == 0 else (2, 2)
			x = ResNet.residual_module(x, filters[i + 1], stride,
				chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

			# loop over the number of layers in the stage
			for j in range(0, stages[i] - 1):
				# apply a ResNet module
				x = ResNet.residual_module(x, filters[i + 1],
					(1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

		# apply BN => ACT => POOL
		x = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = AveragePooling2D((8, 8))(x)

		# softmax classifier
		x = Flatten()(x)
		x = Dense(classes, kernel_regularizer=l2(reg))(x)
		x = Activation("softmax")(x)

		# create the model
		model = Model(inputs, x, name="resnet")

		# return the constructed network architecture
		return model

In [9]:

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
import numpy as np
import requests
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
EPOCHS = 50
INIT_LR = 1e-1
BS=128
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=INIT_LR,
    decay_steps=10000,
    decay_rate=0.96)
opt = SGD(learning_rate=lr_schedule, momentum=0.9)

# Build the model
with tf.device('/device:GPU:0'):
    model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3),
    (64, 64, 128, 256), reg=0.0005)

    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, random_state=42)
    H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,epochs=EPOCHS,
    class_weight=classWeight,
    verbose=1)

Epoch 1/50
2765/2765 [==============================] - 415s 147ms/step - loss: 2.5584 - accuracy: 0.8415 - val_loss: 1.3271 - val_accuracy: 0.7946
Epoch 2/50
2765/2765 [==============================] - 493s 178ms/step - loss: 1.5793 - accuracy: 0.8987 - val_loss: 0.8847 - val_accuracy: 0.8734
Epoch 3/50
2765/2765 [==============================] - 461s 167ms/step - loss: 1.4675 - accuracy: 0.9055 - val_loss: 0.9651 - val_accuracy: 0.8585
Epoch 4/50
2765/2765 [==============================] - 461s 167ms/step - loss: 1.4006 - accuracy: 0.9128 - val_loss: 0.8971 - val_accuracy: 0.8629
Epoch 5/50
2765/2765 [==============================] - 463s 168ms/step - loss: 1.3444 - accuracy: 0.9160 - val_loss: 0.9532 - val_accuracy: 0.8367
Epoch 6/50
2765/2765 [==============================] - 464s 168ms/step - loss: 1.3169 - accuracy: 0.9183 - val_loss: 0.7306 - val_accuracy: 0.9397
Epoch 7/50
2765/2765 [==============================] - 463s 168ms/step - loss: 1.2957 - accuracy: 0.9196 - val_

In [10]:
model.save('resnet-az.h5',save_format=".h5")

In [11]:
#trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, random_state=42)
labelNames = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

predictions = model.predict(testX)


print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), labels=np.arange(72), target_names=labelNames))



2766/2766 [==============================] - 29s 10ms/step
              precision    recall  f1-score   support

           0       0.10      0.79      0.18      1395
           1       0.97      0.97      0.97      1590
           2       0.57      0.98      0.72      1395
           3       0.90      0.93      0.92      1441
           4       0.67      0.98      0.80      1384
           5       0.24      0.95      0.38      1265
           6       0.99      0.78      0.87      1341
           7       0.87      0.98      0.92      1519
           8       0.81      0.99      0.89      1368
           9       0.96      0.95      0.96      1383
           A       1.00      0.98      0.99      2721
           B       0.99      0.94      0.97      1733
           C       0.99      0.94      0.97      4637
           D       0.91      0.96      0.93      2067
           E       0.97      0.98      0.97      2297
           F       0.99      0.96      0.97       215
           G       0.7

C:\Users\lahbi\anaconda3\envs\PFE\lib\site-packages\sklearn\metrics\_classification.py:2326: UserWarning: labels size, 72, does not match size of target_names, 36
  warnings.warn(
C:\Users\lahbi\anaconda3\envs\PFE\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lahbi\anaconda3\envs\PFE\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lahbi\anaconda3\envs\PFE\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model("resnet.h5")

output = ""
images = []
test=
for i in np.random.choice(np.arange(0, len(testY)), size=(49,)):
    probs = model.predict(testX[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    print(labelNames[prediction[0]])
    label = labelNames[prediction[0]]
    output += label
    image = (testX[i] * 255).astype("uint8")
    color = (0, 255, 0)
    if prediction[0] != np.argmax(testY[i]):
        color = (0, 0, 255)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
                color, 2)
    images.append(image)


montage = np.zeros((7 * 96, 7 * 96, 3), dtype="uint8")

for i in range(0, 49):
    row = int(i / 7)
    col = i % 7
    montage[row * 96:(row + 1) * 96, col * 96:(col + 1) * 96] = images[i]

cv2.imshow("Montage", montage)
cv2.waitKey(0)



In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tensorflow.keras.models import load_model
import imutils
import tensorflow as tf

# Create a Tkinter root window (hidden)
root = tk.Tk()
root.withdraw()

# Ask the user to select an image file using a file dialog box
file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])


model = load_model("resnet.h5")
labelNames = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Read image
image = cv2.imread(file_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold image
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

# Find contours in the image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours left-to-right
contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

output = ""
for contour in contours:
    # Get bounding box of contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Extract character image from thresholded image
    roi = thresh[y:y+h, x:x+w]
    
    # Resize character image to 32x32
    roi = cv2.resize(roi, (32, 32), interpolation=cv2.INTER_LINEAR)
    
    # Add channel dimension to character image
    roi = np.expand_dims(roi, axis=-1)
    
    # Normalize pixel values to range [0, 1]
    roi = roi.astype("float") / 255.0
    
    # Make prediction using OCR model
    with tf.device('/device:GPU:0'):
        probs = model.predict(np.array([roi]))
        prediction = probs.argmax(axis=1)
        label = labelNames[prediction[0]]
        output += label
        cv2.putText(image, str(label[0]), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# Display the image with recognized text
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Print recognized text
print("Recognized text:", output)

